In [6]:
%pip install opencv-python
%pip install Pillow


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch
import ffmpeg
from transnet import TransNetV2
from inference import predict_video
import cv2
import numpy as np
import torch
import os
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import ttk

model = TransNetV2()
state_dict = torch.load("transnetv2-pytorch-weights.pth")
model.load_state_dict(state_dict)

<All keys matched successfully>

In [9]:
with torch.no_grad():
    # shape: batch dim x video frames x frame height x frame width x RGB (not BGR) channels
    scenes = predict_video('../../Test_dataset/00112.mp4', model)
    print("Detected scenes:", scenes)
    #visualize_detection('../../Dataset/00100.mp4', scenes,'./')

Trying for ../../Test_dataset/00112.mp4
[TransNetV2] Processing video frames 7676/7676Detected scenes: [[   0   71]
 [  73  134]
 [ 135  217]
 [ 218  302]
 [ 303  366]
 [ 367  460]
 [ 461  500]
 [ 501  533]
 [ 534  567]
 [ 568  616]
 [ 617  665]
 [ 666  698]
 [ 699  765]
 [ 766  795]
 [ 796  831]
 [ 832  865]
 [ 866  897]
 [ 898  934]
 [ 935  983]
 [ 984 1028]
 [1029 1126]
 [1127 1157]
 [1158 1195]
 [1196 1232]
 [1233 1263]
 [1264 1288]
 [1289 1468]
 [1469 1492]
 [1493 1528]
 [1529 1560]
 [1561 1630]
 [1631 1658]
 [1659 1694]
 [1695 1728]
 [1729 1820]
 [1821 1838]
 [1839 1853]
 [1854 1884]
 [1885 1926]
 [1927 1947]
 [1948 2004]
 [2005 2009]
 [2010 2019]
 [2020 2050]
 [2051 2091]
 [2092 2113]
 [2114 2154]
 [2155 2169]
 [2170 2195]
 [2196 2236]
 [2237 2277]
 [2278 2298]
 [2299 2414]
 [2415 2464]
 [2465 2500]
 [2501 2517]
 [2518 2546]
 [2547 2599]
 [2600 2641]
 [2642 2724]
 [2725 2763]
 [2764 2795]
 [2796 2831]
 [2832 2857]
 [2858 2877]
 [2878 2899]
 [2900 2938]
 [2939 2959]
 [2960 2982]


In [19]:


def extract_frame(video_path, frame_idx):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        return None
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return Image.fromarray(frame_rgb)

def visualize_shots_scrollable(video_path, scenes_data):
    if isinstance(scenes_data, torch.Tensor):
        scenes_data = scenes_data.cpu().numpy()
    elif isinstance(scenes_data, list):
        scenes_data = np.array(scenes_data)

    root = tk.Tk()
    root.title("Shot Boundary Viewer")
    def on_closing():
        root.quit()
        root.destroy()

    root.protocol("WM_DELETE_WINDOW", on_closing)

    main_frame = ttk.Frame(root)
    main_frame.pack(fill=tk.BOTH, expand=True)

    canvas = tk.Canvas(main_frame)
    scrollbar = ttk.Scrollbar(main_frame, orient=tk.VERTICAL, command=canvas.yview)
    scrollable_frame = ttk.Frame(canvas)

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(
            scrollregion=canvas.bbox("all")
        )
    )

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)

    canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    for idx, (start, end) in enumerate(scenes_data):
        start_img = extract_frame(video_path, int(start))
        end_img = extract_frame(video_path, int(end))

        if start_img is None or end_img is None:
            continue

        start_img_tk = ImageTk.PhotoImage(start_img.resize((160, 90)))
        end_img_tk = ImageTk.PhotoImage(end_img.resize((160, 90)))

        pair_frame = ttk.Frame(scrollable_frame, padding=10)
        pair_frame.pack(fill=tk.X)

        ttk.Label(pair_frame, text=f"Shot {idx+1}: [{start}-{end}]").pack(anchor="w")

        images_frame = ttk.Frame(pair_frame)
        images_frame.pack()

        ttk.Label(images_frame, image=start_img_tk).pack(side=tk.LEFT, padx=5)
        ttk.Label(images_frame, image=end_img_tk).pack(side=tk.LEFT, padx=5)

        # Store reference to avoid garbage collection
        pair_frame.start_img_tk = start_img_tk
        pair_frame.end_img_tk = end_img_tk

    root.mainloop()

if __name__ == '__main__':
    scenes_np = np.array(scenes)
    print(os.path.exists("00112.mp4"))
    visualize_shots_scrollable("00112.mp4", scenes_np)



True


KeyboardInterrupt: 